In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
from collections import Counter
import math
import datetime as datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import statistics
import glob
import ipyplot
import numpy as np
from functions import data_preprocess_functions


In [ ]:
main_plot_path = "idl_plots/output_5min_multi_subtraction_removebi_flux0.500.751.00_pap3.02.01.1/"


data_filename = main_plot_path + 'data/fulldata_20160101_to_20201231.csv'
df = data_preprocess_functions.preprocess_data(pd.read_csv(data_filename))

In [ ]:
index2016 = (df['year'] == '2016')
index2017 = (df['year'] == '2017')
index2018 = (df['year'] == '2018')
index2020 = (df['year'] == '2020')

In [ ]:
df['flag'].value_counts()

In [ ]:
print(statistics.median(abs(df['kp'])))
print(statistics.median(abs(df['bx'])))
print(statistics.median(abs(df['v_par_all'])))
print(statistics.median(abs(df['v_perp_all'])))
print(statistics.median(abs(df['swv'])))
print(statistics.median(abs(df['swp'])))

In [ ]:

index = (df['year'] == '2016') 
fig = px.scatter_3d(df, x='GSM_X', y='GSM_Y', z='GSM_Z', color='region')
fig.update_traces(marker_size = 1)
fig.show()

In [ ]:
index = (df['GSM_Z'] > -20) 
fig = px.scatter_3d(df.loc[index,:], x='GSM_X', y='GSM_Y', z='GSM_Z', color='year')
fig.update_traces(marker_size = 1)
fig.show()

In [ ]:
index =   (df['region'] == 'bl')& ((df['year'] == '2017') | (df['year'] == '2018'))
fig = px.scatter_3d(df.loc[index,:], x='GSM_X', y='GSM_Y', z='GSM_Z', color='year')
fig.update_traces(marker_size = 1)
fig.show()

In [ ]:
index =   (df['region'] == 'bl') &  ((df['MLT'] < 6) |  (df['MLT'] > 18)) &(df['GSM_Z'] < 0) & (df['year'] != '2016')&(df['GSM_Z'] > -3)
fig = px.scatter_3d(df.loc[index,:], x='GSM_X', y='GSM_Y', z='GSM_Z', color='energy')
fig.update_traces(marker_size = 1)

fig.show()

In [ ]:
index =   (df['region'] == 'ps') 
fig = px.scatter_3d(df.loc[index,:], x='GSM_X', y='GSM_Y', z='GSM_Z', color='energy')
fig.update_traces(marker_size = 1)

fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=4, subplot_titles=("GSM_X", "GSM_Y", "GSM_Z", "DIST"))

index =  (df['year'] == '2016') & (df['GSM_X'] < 0)& (df['GSM_Y'] < 4) &(df['GSM_Y'] > -4) & (df['GSM_X'] <0) &(df['GSM_X'] > -15) &(df['DIST'] <10) & (df['GSM_Z'] > -3.5) &(df['GSM_Z'] < 3.5)&(df['DIST'] >6)

fig.add_trace(go.Histogram(x=df.loc[index,'GSM_X']), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Y']), row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Z']), row=1, col=3)
fig.add_trace(go.Histogram(x=df.loc[index,'DIST']), row=1, col=4)

index =  ((df['year'] == '2017') | (df['year'] == '2018')|(df['year'] == '2019')|(df['year'] == '2020'))  & (df['GSM_Y'] < 4) &(df['GSM_Y'] > -4) & (df['GSM_X'] < 0) &(df['GSM_X'] > -15) &(df['DIST'] <10) &(df['DIST'] >6)& (df['GSM_Z'] > -3.5) &(df['GSM_Z'] < 3.5)

fig.add_trace(go.Histogram(x=df.loc[index,'GSM_X']), row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Y']), row=2, col=2)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Z']), row=2, col=3)
fig.add_trace(go.Histogram(x=df.loc[index,'DIST']), row=2, col=4) 

fig.update_layout(showlegend=False, title='')

fig.show()

In [ ]:
index =   (df['GSM_X'] < 0)& (df['GSM_Y'] < 8) &(df['GSM_Y'] > -8) & (df['GSM_X'] <0) &(df['GSM_X'] > -15) &(df['DIST'] <10) & (df['GSM_Z'] > -3.5) &(df['GSM_Z'] < 0)&(df['DIST'] >6)

fig = px.scatter_3d(df.loc[index,:], x='GSM_X', y='GSM_Y', z='GSM_Z', color='energy',size_max=1)
fig.update_traces(marker_size = 1)

fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=4, subplot_titles=("GSM_X", "GSM_Y", "GSM_Z", "DIST"))

index =  (df['region'] == 'lobe')

fig.add_trace(go.Histogram(x=df.loc[index,'GSM_X']), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Y']), row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Z']), row=1, col=3)
fig.add_trace(go.Histogram(x=df.loc[index,'DIST']), row=1, col=4)

index =  (df['region'] == 'lobe') & (df['GSM_X'] < -10)

fig.add_trace(go.Histogram(x=df.loc[index,'GSM_X']), row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Y']), row=2, col=2)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Z']), row=2, col=3)
fig.add_trace(go.Histogram(x=df.loc[index,'DIST']), row=2, col=4) 

fig.update_layout(showlegend=False, title='')

fig.show()

In [ ]:
#index = (df['eflux'] > 0) & (df['GSM_X'] < -1)  & (df['compression_mode']) & (df['year'] == 2017) & (df['region'] == 'bl') & (df['energy_int'] < 40)
# property_to_plot = 'dist'

fig = make_subplots(rows=2, cols=4, subplot_titles=("GSM_X", "GSM_Y", "GSM_Z", "DIST"))

index =  (df['region'] == 'lobe') & (df['GSM_X'] < -10)

fig.add_trace(go.Histogram(x=df.loc[index,'GSM_X']), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Y']), row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Z']), row=1, col=3)
fig.add_trace(go.Histogram(x=df.loc[index,'DIST']), row=1, col=4)

index =  (df['region'] == 'lobe') & (df['flux'] > 0) & (df['GSM_X'] < -10)

fig.add_trace(go.Histogram(x=df.loc[index,'GSM_X']), row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Y']), row=2, col=2)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Z']), row=2, col=3)
fig.add_trace(go.Histogram(x=df.loc[index,'DIST']), row=2, col=4) 

fig.update_layout(showlegend=False, title='Lobe: all observations vs O+ events')

fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=4, subplot_titles=("GSM_X", "GSM_Y", "GSM_Z", "DIST"))

index =  (df['region'] == 'bl') & (df['GSM_X'] < -10)

fig.add_trace(go.Histogram(x=df.loc[index,'GSM_X']), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Y']), row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Z']), row=1, col=3)
fig.add_trace(go.Histogram(x=df.loc[index,'DIST']), row=1, col=4)

index =  (df['region'] == 'bl') & (df['flux'] > 0) & (df['GSM_X'] < -10)

fig.add_trace(go.Histogram(x=df.loc[index,'GSM_X']), row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Y']), row=2, col=2)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Z']), row=2, col=3)
fig.add_trace(go.Histogram(x=df.loc[index,'DIST']), row=2, col=4) 

fig.update_layout(showlegend=False, title='bl: all observations vs O+ events')

fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=4, subplot_titles=("GSM_X", "GSM_Y", "GSM_Z", "DIST"))

index =  (df['region'] == 'ps') & (df['GSM_X'] < -10)

fig.add_trace(go.Histogram(x=df.loc[index,'GSM_X']), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Y']), row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Z']), row=1, col=3)
fig.add_trace(go.Histogram(x=df.loc[index,'DIST']), row=1, col=4)

index =  (df['region'] == 'ps') & (df['flux'] > 0) & (df['GSM_X'] < -10)

fig.add_trace(go.Histogram(x=df.loc[index,'GSM_X']), row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Y']), row=2, col=2)
fig.add_trace(go.Histogram(x=df.loc[index,'GSM_Z']), row=2, col=3)
fig.add_trace(go.Histogram(x=df.loc[index,'DIST']), row=2, col=4) 

fig.update_layout(showlegend=False, title='ps: all observations vs O+ events')

fig.show()

In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Kp 2016", "Kp 2017", "Kp 2018", "Kp 2020"))

fig.add_trace(go.Histogram(x=df.loc[index2016, 'kp'], xbins=dict(start=0,end=7, size=1)), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2017, 'kp'], xbins=dict(start=0,end=7, size=1)),row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index2018, 'kp'], xbins=dict(start=0,end=7, size=1)) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2020, 'kp'], xbins=dict(start=0,end=7, size=1)) ,row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("bx 2016", "bx 2017", "bx 2018", "bx 2020"))

fig.add_trace(go.Histogram(x=abs(df.loc[index2016, 'bx']), xbins=dict(start=0,end=100, size=1)), row=1, col=1)
fig.add_trace(go.Histogram(x=abs(df.loc[index2017, 'bx']), xbins=dict(start=0,end=100, size=1)),row=1, col=2)
fig.add_trace(go.Histogram(x=abs(df.loc[index2018, 'bx']), xbins=dict(start=0,end=100, size=1)) ,row=2, col=1)
fig.add_trace(go.Histogram(x=abs(df.loc[index2020, 'bx']), xbins=dict(start=0,end=100, size=1)) ,row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("swv 2016", "swv 2017", "swv 2018", "swv 2020"))

fig.add_trace(go.Histogram(x=df.loc[index2016, 'swv'], xbins=dict(start=0,end=900, size=10)), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2017, 'swv'], xbins=dict(start=0,end=900, size=10)) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index2018, 'swv'], xbins=dict(start=0,end=900, size=10)) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2020, 'swv'], xbins=dict(start=0,end=900, size=10)) ,row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("swp 2016", "swp 2017", "swp 2018", "swp 2020"))

fig.add_trace(go.Histogram(x=df.loc[index2016, 'swp'], xbins=dict(start=0,end=20, size=1)), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2017, 'swp'], xbins=dict(start=0,end=20, size=1)) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index2018, 'swp'], xbins=dict(start=0,end=20, size=1)) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2020, 'swp'], xbins=dict(start=0,end=20, size=1)) ,row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("v_perp_all 2016", "v_perp_all 2017", "v_perp_all 2018", "v_perp_all 2020"))

fig.add_trace(go.Histogram(x=df.loc[index2016, 'v_perp_all'], xbins=dict(start=0,end=500, size=10 )), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2017, 'v_perp_all'], xbins=dict(start=0,end=500, size=10 )) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index2018, 'v_perp_all'], xbins=dict(start=0,end=500, size=10 )) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2020, 'v_perp_all'], xbins=dict(start=0,end=500, size=10 )) ,row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:

fig = make_subplots(rows=2, cols=2, subplot_titles=("storm_phase 2016", "storm_phase 2017", "storm_phase 2018", "storm_phase 2020"))
fig.add_trace(go.Histogram(x=df.loc[index2016, 'storm_phase']), row=1, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2017, 'storm_phase']) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df.loc[index2018, 'storm_phase']) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df.loc[index2020, 'storm_phase']) ,row=2, col=2)

fig.update_xaxes(categoryorder='array', categoryarray= ['nonstorm','prestorm','main phase','fast recovery','long recovery'], row=1, col=1)
fig.update_xaxes(categoryorder='array', categoryarray= ['nonstorm','prestorm','main phase','fast recovery','long recovery'], row=1, col=2)
fig.update_xaxes(categoryorder='array', categoryarray= ['nonstorm','prestorm','main phase','fast recovery','long recovery'], row=2, col=1)
fig.update_xaxes(categoryorder='array', categoryarray= ['nonstorm','prestorm','main phase','fast recovery','long recovery'], row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Storm Phases", "Kp Index", "IMF Bz", "Solar wind Pressure"))

fig.add_trace(go.Histogram(x=df['storm_phase']), row=1, col=1)
fig.add_trace(go.Histogram(x=df['kp'], xbins=dict(start=0,end=7.0, size=0.1 )) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df['imfBz']) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df['swp']) ,row=2, col=2)

fig.update_xaxes(categoryorder='array', categoryarray= ['nonstorm','prestorm','main phase','fast recovery','long recovery'], row=1, col=1)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Energy", "Flux", "Pitch Angle", "Region"))

fig.add_trace(go.Histogram(x=df['energy']), row=1, col=1)
fig.add_trace(go.Histogram(x=df['flux'], xbins=dict(start=0,end=200, size=1)) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df['pitch angle']) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df['region']) ,row=2, col=2)

#fig.update_xaxes(type="log", range = [0,5],row=1, col=1)
fig.update_xaxes(type="log", range = [0,2],row=1, col=2)
fig.update_xaxes(range = [1,4],row=2, col=2)

#fig.update_xaxes(type="log", row=1, col=2)
#fig.update_xaxes(type="log", row=2, col=2)

fig.update_layout(showlegend=False)


In [ ]:
fig = make_subplots(rows=2, cols=2, subplot_titles=("Vel_tot", "Density", "Pressure", "H+ density"))

fig.add_trace(go.Histogram(x=df['velocity_o_all']), row=1, col=1)
fig.add_trace(go.Histogram(x=df['density_o_all'], xbins=dict(start=0.01,end=10, size=0.1)) ,row=1, col=2)
fig.add_trace(go.Histogram(x=df['pressure_o_all'], xbins=dict(start=0.001,end=0.01, size=0.001)) ,row=2, col=1)
fig.add_trace(go.Histogram(x=df['density_h_all'], xbins=dict(start=0.01,end=10, size=0.1 )) ,row=2, col=2)

#fig.update_xaxes(type="log", range = [0,5],row=1, col=1)
fig.update_xaxes(type="log", range = [-1,1],row=1, col=2)
#fig.update_xaxes(type="log", range = [-4,-2],row=2, col=1)
fig.update_xaxes(type="log", range = [-1,1],row=2, col=2)

#fig.update_xaxes(type="log", row=1, col=2)
#fig.update_xaxes(type="log", row=2, col=2)

fig.update_layout(showlegend=False)
